# Import the things

In [1]:
# Standard
import pandas as pd
import numpy as np
import math
import datetime as dt
from itertools import product
from itertools import combinations
import operator
import time
import flask

# Input Variables

## Stable Variables

In [318]:
min_wake_window = pd.Timedelta('150 minutes')
max_wake_window = pd.Timedelta('3 hours')

min_nap_length = pd.Timedelta('30 minutes')
max_nap_length = pd.Timedelta('2 hours')

earliest_bedtime = pd.Timestamp(
    tz='US/Pacific',
    year=pd.Timestamp.now(tz='US/Pacific').year, 
    month=pd.Timestamp.now(tz='US/Pacific').month, 
    day=pd.Timestamp.now(tz='US/Pacific').day, 
    hour=19,
    minute=0,
    )
latest_bedtime = pd.Timestamp(
    tz='US/Pacific',
    year=pd.Timestamp.now(tz='US/Pacific').year, 
    month=pd.Timestamp.now(tz='US/Pacific').month, 
    day=pd.Timestamp.now(tz='US/Pacific').day, 
    hour=21,
    minute=0,
    )

possible_num_naps = [0,1]
round_times_by = '15 minutes'
round_times_by_num = int(round_times_by.split()[0])

## Today's Variables

In [319]:
last_nap_still_ongoing = False
last_nap_start_time = pd.Timestamp(
    tz='US/Pacific',
    year=pd.Timestamp.now(tz='US/Pacific').year, 
    month=pd.Timestamp.now(tz='US/Pacific').month, 
    day=pd.Timestamp.now(tz='US/Pacific').day, 
    hour=14,
    minute=20,
    )

In [320]:
last_nap_end_time = pd.Timestamp(
    tz='US/Pacific',
    year=pd.Timestamp.now(tz='US/Pacific').year, 
    month=pd.Timestamp.now(tz='US/Pacific').month, 
    day=pd.Timestamp.now(tz='US/Pacific').day, 
    hour=15,
    minute=0,
    )

# Calculator

In [321]:
now_time =  pd.Timestamp.now(tz='US/Pacific')

## Generating possibilities

In [322]:
# Generating possible nap, wake window lengths, and when to wake wake from ongoing nap
possible_nap_lengths = []
i = min_nap_length
while i <= max_nap_length:
    possible_nap_lengths.append(i)
    i += pd.Timedelta(round_times_by)

if last_nap_still_ongoing == True:
    possible_ongoing_nap_length = []
    possible_nap_end = max(now_time, last_nap_start_time + min_nap_length)
    
    while possible_nap_end <= last_nap_start_time + max_nap_length:
        possible_ongoing_nap_length.append(((possible_nap_end-last_nap_start_time),))
        
        num_seconds = possible_nap_end.minute*60
        delta = math.ceil(num_seconds / round_times_by_num*60) * round_times_by_num*60 - num_seconds
        possible_nap_end = possible_nap_end + dt.timedelta(seconds=delta)
        
        possible_nap_end += pd.Timedelta(round_times_by)

possible_wake_window_lengths = []
i = min_wake_window
while i <= max_wake_window:
    possible_wake_window_lengths.append(i)
    i += pd.Timedelta(round_times_by)

In [323]:
## Creating combos of naps and wake window lengths
## Number of naps here includes the ongoing nap
nap_possibilities = pd.DataFrame()

for num_nap in possible_num_naps:
    print('Checking for '+str(num_nap)+' naps')
    nap_possibility = pd.DataFrame()
        
    print('Generating nap sequences and wake window sequences')
    if last_nap_still_ongoing == True:
        nap_combos = list(product(possible_nap_lengths,repeat=max(num_nap-1,0)))
        ww_combos = list(product(possible_wake_window_lengths,repeat=max(num_nap-1,0)+1))
    elif last_nap_still_ongoing == False:
        nap_combos = list(product(possible_nap_lengths,repeat=num_nap))
        ww_combos = list(product(possible_wake_window_lengths,repeat=num_nap+1))
    
    print('Generating nap+wake window sequences')
    ## Nap sequence includes the ongoing nap
    if last_nap_still_ongoing == True:
        all_combos_tuple = list(product(possible_ongoing_nap_length, nap_combos, ww_combos))
        nap_sequence = [list(i[0])+ list(i[1]) for i in all_combos_tuple]
        ww_sequence = [list(i[2]) for i in all_combos_tuple]
    elif last_nap_still_ongoing == False:
        all_combos_tuple = list(product(nap_combos, ww_combos))
        nap_sequence = [list(i[0]) for i in all_combos_tuple]
        ww_sequence = [list(i[1]) for i in all_combos_tuple]
    
    print('Sequences created. Combining sequences together.')
    all_combos = []
    for element in all_combos_tuple:
        combos_summed = [i for sub in element for i in sub]
        all_combos.append(combos_summed)
    
    print(str(len(all_combos))+' combinations created. Summing length of each combo.')
    total_lengths = []
    for combo in all_combos:
        total_lengths.append(pd.Series(combo).sum())
    
    print('Creating dataframe')
    nap_possibility['nap_sequence'] = nap_sequence
    nap_possibility['ww_sequence'] = ww_sequence
    nap_possibility['total_time'] = total_lengths
    nap_possibility['num_naps'] = num_nap
    
    nap_possibilities = nap_possibilities.append(nap_possibility)

Checking for 0 naps
Generating nap sequences and wake window sequences
Generating nap+wake window sequences
Sequences created. Combining sequences together.
3 combinations created. Summing length of each combo.
Creating dataframe
Checking for 1 naps
Checking for 0 naps
Generating nap sequences and wake window sequences
Generating nap+wake window sequences
Sequences created. Combining sequences together.
3 combinations created. Summing length of each combo.
Creating dataframe
Checking for 1 naps
Generating nap sequences and wake window sequences
Generating nap+wake window sequences
Sequences created. Combining sequences together.
63 combinations created. Summing length of each combo.
Generating nap sequences and wake window sequences
Generating nap+wake window sequences
Sequences created. Combining sequences together.
63 combinations created. Summing length of each combo.
Creating dataframe
Creating dataframe


In [324]:
nap_possibilities.head()

,nap_sequence,ww_sequence,total_time,num_naps
0,[],[0 days 02:30:00],02:30:00,0
1,[],[0 days 02:45:00],02:45:00,0
2,[],[0 days 03:00:00],03:00:00,0
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1


,nap_sequence,ww_sequence,total_time,num_naps
0,[],[0 days 02:30:00],02:30:00,0
1,[],[0 days 02:45:00],02:45:00,0
2,[],[0 days 03:00:00],03:00:00,0
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1


## Filtering for valid possibilities

### Filter based on bedtime

In [325]:
valid_possibilities = nap_possibilities

if last_nap_still_ongoing == True:
    valid_possibilities['bedtime'] = last_nap_start_time + valid_possibilities['total_time']
elif last_nap_still_ongoing == False:
    valid_possibilities['bedtime'] = last_nap_end_time + valid_possibilities['total_time']

valid_possibilities = valid_possibilities[valid_possibilities['bedtime'] >=  earliest_bedtime]
valid_possibilities = valid_possibilities[valid_possibilities['bedtime'] <=  latest_bedtime]

In [326]:
valid_possibilities.head()

,nap_sequence,ww_sequence,total_time,num_naps,bedtime
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1,2020-12-17 20:30:00-08:00
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1,2020-12-17 20:45:00-08:00
2,[0 days 00:30:00],"[0 days 02:30:00, 0 days 03:00:00]",06:00:00,1,2020-12-17 21:00:00-08:00
3,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:30:00]",05:45:00,1,2020-12-17 20:45:00-08:00
4,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:45:00]",06:00:00,1,2020-12-17 21:00:00-08:00


,nap_sequence,ww_sequence,total_time,num_naps,bedtime
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1,2020-12-17 20:30:00-08:00
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1,2020-12-17 20:45:00-08:00
2,[0 days 00:30:00],"[0 days 02:30:00, 0 days 03:00:00]",06:00:00,1,2020-12-17 21:00:00-08:00
3,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:30:00]",05:45:00,1,2020-12-17 20:45:00-08:00
4,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:45:00]",06:00:00,1,2020-12-17 21:00:00-08:00


### Create preference based on min naptimes

In [327]:
if last_nap_still_ongoing == True:
    valid_possibilities['last_nap_length'] = [i[0] for i in valid_possibilities['nap_sequence']]
elif last_nap_still_ongoing == False:
    valid_possibilities['last_nap_length'] = last_nap_end_time - last_nap_start_time 
valid_possibilities['next_ww'] = [i[0] for i in valid_possibilities['ww_sequence']]
valid_possibilities['start_next_nap_at'] = last_nap_start_time + valid_possibilities['last_nap_length'] + valid_possibilities['next_ww']

#### Calculating the preferred next nap times

In [328]:
min_nap_length_boolean = []
if last_nap_still_ongoing == True:
    for nap_sequence in valid_possibilities['nap_sequence']:
        min_nap_length_boolean.append(all([nap_sequence[0] == now_time - last_nap_start_time]+[nap == min_nap_length for nap in nap_sequence[1:]]))
elif last_nap_still_ongoing == False:
    for nap_sequence in valid_possibilities['nap_sequence']:
        min_nap_length_boolean.append(all([nap == min_nap_length for nap in nap_sequence]))
valid_possibilities['preferred'] = min_nap_length_boolean
preferred_map = valid_possibilities[['start_next_nap_at','preferred']][valid_possibilities['preferred'] == True].drop_duplicates()

#### Merge the preferences back in

In [329]:
valid_possibilities = valid_possibilities.drop(['preferred'], axis=1)
valid_possibilities = valid_possibilities.merge(preferred_map, on='start_next_nap_at', how='left')
valid_possibilities = valid_possibilities.fillna(False)

In [330]:
valid_possibilities.head()

,nap_sequence,ww_sequence,total_time,num_naps,bedtime,last_nap_length,next_ww,start_next_nap_at,preferred
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1,2020-12-17 20:30:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1,2020-12-17 20:45:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
2,[0 days 00:30:00],"[0 days 02:30:00, 0 days 03:00:00]",06:00:00,1,2020-12-17 21:00:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
3,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:30:00]",05:45:00,1,2020-12-17 20:45:00-08:00,00:40:00,02:45:00,2020-12-17 17:45:00-08:00,True
4,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:45:00]",06:00:00,1,2020-12-17 21:00:00-08:00,00:40:00,02:45:00,2020-12-17 17:45:00-08:00,True


,nap_sequence,ww_sequence,total_time,num_naps,bedtime,last_nap_length,next_ww,start_next_nap_at,preferred
0,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:30:00]",05:30:00,1,2020-12-17 20:30:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
1,[0 days 00:30:00],"[0 days 02:30:00, 0 days 02:45:00]",05:45:00,1,2020-12-17 20:45:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
2,[0 days 00:30:00],"[0 days 02:30:00, 0 days 03:00:00]",06:00:00,1,2020-12-17 21:00:00-08:00,00:40:00,02:30:00,2020-12-17 17:30:00-08:00,True
3,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:30:00]",05:45:00,1,2020-12-17 20:45:00-08:00,00:40:00,02:45:00,2020-12-17 17:45:00-08:00,True
4,[0 days 00:30:00],"[0 days 02:45:00, 0 days 02:45:00]",06:00:00,1,2020-12-17 21:00:00-08:00,00:40:00,02:45:00,2020-12-17 17:45:00-08:00,True


## Making the output pretty

In [331]:
## Reformating the time columns to be more readable
cleaned_possibilities = valid_possibilities
cleaned_possibilities['bedtime'] = cleaned_possibilities['bedtime'].dt.strftime("%-I:%M%p")

pretty_nap_sequence = []
for row in cleaned_possibilities['nap_sequence']:
    pretty_nap_sequence.append([time.strftime("%H:%M:%S", time.gmtime(s.seconds)) for s in row])
cleaned_possibilities['pretty_nap_sequence'] = pretty_nap_sequence

pretty_ww_sequence = []
for row in cleaned_possibilities['ww_sequence']:
    pretty_ww_sequence.append([time.strftime("%H:%M:%S", time.gmtime(s.seconds)) for s in row])
cleaned_possibilities['pretty_ww_sequence'] = pretty_ww_sequence

In [332]:
## Again, how_many_more_naps is how many additional more naps
cleaned_possibilities['start_next_nap_at'] = cleaned_possibilities['start_next_nap_at'].dt.strftime("%-I:%M%p")

if last_nap_still_ongoing == True:
    cleaned_possibilities['how_many_more_naps'] = [max(i-1, 0) for i in cleaned_possibilities['num_naps']]
    
    cleaned_possibilities['end_ongoing_nap_at'] = last_nap_start_time + cleaned_possibilities['last_nap_length']
    cleaned_possibilities['end_ongoing_nap_at'] = cleaned_possibilities['end_ongoing_nap_at'].dt.strftime("%-I:%M%p")
        
    cleaned_possibilities = cleaned_possibilities[[
        'end_ongoing_nap_at',
        'how_many_more_naps',
        'start_next_nap_at',
        'bedtime',
        'pretty_nap_sequence',
        'pretty_ww_sequence',
        'preferred'
        ]]
elif last_nap_still_ongoing == False:
    cleaned_possibilities['how_many_more_naps'] = cleaned_possibilities['num_naps']
            
    cleaned_possibilities = cleaned_possibilities[[
        'how_many_more_naps',
        'start_next_nap_at',
        'bedtime',
        'pretty_nap_sequence',
        'pretty_ww_sequence',
        'preferred'
        ]]

In [333]:
cleaned_possibilities['pretty_nap_sequence'] = cleaned_possibilities['pretty_nap_sequence'].astype(str)
cleaned_possibilities.sort_values(by=['preferred','start_next_nap_at', 'pretty_nap_sequence']).reset_index(drop=True)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,how_many_more_naps,start_next_nap_at,bedtime,pretty_nap_sequence,pretty_ww_sequence,preferred
0,1,5:30PM,8:30PM,['00:30:00'],"[02:30:00, 02:30:00]",True
1,1,5:30PM,8:45PM,['00:30:00'],"[02:30:00, 02:45:00]",True
2,1,5:30PM,9:00PM,['00:30:00'],"[02:30:00, 03:00:00]",True
3,1,5:30PM,8:45PM,['00:45:00'],"[02:30:00, 02:30:00]",True
4,1,5:30PM,9:00PM,['00:45:00'],"[02:30:00, 02:45:00]",True
5,1,5:30PM,9:00PM,['01:00:00'],"[02:30:00, 02:30:00]",True
6,1,5:45PM,8:45PM,['00:30:00'],"[02:45:00, 02:30:00]",True
7,1,5:45PM,9:00PM,['00:30:00'],"[02:45:00, 02:45:00]",True
8,1,5:45PM,9:00PM,['00:45:00'],"[02:45:00, 02:30:00]",True
9,1,6:00PM,9:00PM,['00:30:00'],"[03:00:00, 02:30:00]",True


,how_many_more_naps,start_next_nap_at,bedtime,pretty_nap_sequence,pretty_ww_sequence,preferred
0,1,5:30PM,8:30PM,['00:30:00'],"[02:30:00, 02:30:00]",True
1,1,5:30PM,8:45PM,['00:30:00'],"[02:30:00, 02:45:00]",True
2,1,5:30PM,9:00PM,['00:30:00'],"[02:30:00, 03:00:00]",True
3,1,5:30PM,8:45PM,['00:45:00'],"[02:30:00, 02:30:00]",True
4,1,5:30PM,9:00PM,['00:45:00'],"[02:30:00, 02:45:00]",True
5,1,5:30PM,9:00PM,['01:00:00'],"[02:30:00, 02:30:00]",True
6,1,5:45PM,8:45PM,['00:30:00'],"[02:45:00, 02:30:00]",True
7,1,5:45PM,9:00PM,['00:30:00'],"[02:45:00, 02:45:00]",True
8,1,5:45PM,9:00PM,['00:45:00'],"[02:45:00, 02:30:00]",True
9,1,6:00PM,9:00PM,['00:30:00'],"[03:00:00, 02:30:00]",True
